# TFM processing to dataset npy
Example workflow converting raw bead images to the 8-channel `.npy` files expected by `CellDataset`.


In [ ]:
from TFM_main.TFM_Image_registration import TFM_Image_registration
from TFM_main.TFM_displacement_tools import TFM_optical_flow
from TFM_main.TFM_tools import TFM_calculation, cellmask_threshold
import skimage.io as io
import numpy as np, os


## 1. Register images and compute displacements
This assumes you have `*_reference.tif` and a time-lapse stack in the current directory.

In [ ]:
TFM_Image_registration(flatfield_correct=False)
TFM_optical_flow()


## 2. Calculate traction forces and create masks

In [ ]:
TFM_calculation(regparam=1e-16)
cellmask, forcemask, _ = cellmask_threshold('zyxin_registered.tif', plot_figure=False)


## 3. Stack channels and save `.npy` files
Replace `zyxin_registered.tif` and `actin_registered.tif` with your fluorescence channels.

In [ ]:
zyx_stack = io.imread('zyxin_registered.tif')
act_stack = io.imread('actin_registered.tif')
for i in range(zyx_stack.shape[0]):
    ux = io.imread(f'displacement_files/disp_u_{i:03d}.tif')
    uy = io.imread(f'displacement_files/disp_v_{i:03d}.tif')
    fx = io.imread(f'traction_files/fx_{i+1:03d}.tif')
    fy = io.imread(f'traction_files/fy_{i+1:03d}.tif')
    frame = np.stack([ux, uy, fx, fy, cellmask[i], forcemask[i], zyx_stack[i], act_stack[i]])
    np.save(f'frame_{i}.npy', frame)
